In [1]:
dataset = 'dataset/'
import os

## Berikut adalah kodingan untuk klasifikasi menggunakan algoritma KNN

In [3]:
import numpy as np
import cv2
import os
import pickle
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import mean_squared_error
import joblib
import json

# Path to the dataset directory
data_dir = dataset

# List all image files in the dataset directory
image_files = [os.path.join(data_dir, f) for f in os.listdir(data_dir) if f.endswith('.jpg')]

# Load images and extract features
X = []
y = []
for image_file in image_files:
    img = cv2.imread(image_file)
    hsv_img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    hsv_hist = cv2.calcHist([hsv_img], [0, 1, 2], None, [8, 2, 2], [0, 180, 0, 256, 0, 256])
    hsv_hist = cv2.normalize(hsv_hist, hsv_hist).flatten()
    X.append(hsv_hist)
    label = image_file.split('/')[-1].split('_')[0]
    # ubah label dari `mata ikan gak segar` menjadi 0 dan` mata ikan segar` menjadi 1
    # print(label)
    if label == 'tidak segar':
        y.append(0)
    if label == 'segar':
        y.append(1)

# Convert data to numpy arrays
X = np.array(X)
y = np.array(y)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# KNN classification
knn = KNeighborsClassifier(n_neighbors=50)
knn.fit(X_train, y_train)

# Save KNN model to file
pickle.dump(knn, open('KNN.pkl', 'wb'))

#knn.save('KNN.h5')

# Evaluate KNN model
knn_preds = knn.predict(X_test)
knn_rmse = mean_squared_error(y_test, knn_preds, squared=False)
print('KNN RMSE:', knn_rmse)

# tampikan akurasinya
print('Akurasi KNN:', knn.score(X_test, y_test))

joblib.dump(knn, 'knn_model.pkl')
with open('knn_model.json', 'w') as f:
    f.write(json.dumps(joblib.load('knn_model.pkl').get_params()))

KNN RMSE: 0.0
Akurasi KNN: 1.0


## Berikut adalah kodingan untuk klasifikasi menggunakan algoritma CNN

In [3]:
import numpy as np
import cv2
import os
import pickle
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Reshape
from keras.optimizers import Adam
from sklearn.metrics import mean_squared_error
dataset = 'dataset/'
import os

# Path to the dataset directory
data_dir = dataset

# List all image files in the dataset directory
image_files = [os.path.join(data_dir, f) for f in os.listdir(data_dir) if f.endswith('.jpg')]

# Load images and extract features
X = []
y = []
for image_file in image_files:
    img = cv2.imread(image_file)
    #hsv_img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    #hsv_hist = cv2.calcHist([hsv_img], [0, 1, 2], None, [8, 2, 2], [0, 180, 0, 256, 0, 256])
    #hsv_hist = cv2.normalize(hsv_hist, hsv_hist).flatten()
    X.append(list(img))
    label = image_file.split('/')[-1].split('_')[0]
    y.append(label)

# Convert data to numpy arrays
X = np.array(X)
y = np.array(y)

# ubah label dari `mata ikan gak segar` menjadi 0 dan` mata ikan segar` menjadi 1
y[y == 'tidak segar'] = 0
y[y == 'segar'] = 1

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Define the CNN model
model = Sequential()
model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid',dtype='float32'))

# Compile the CNN model
model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

unique_labels = np.unique(y_train)
print(unique_labels)
label_map = {label: i for i, label in enumerate(unique_labels)}
y_train = np.array([label_map[label] for label in y_train])

unique_labels = np.unique(y_test)
label_map = {label: i for i, label in enumerate(unique_labels)}
y_test = np.array([label_map[label] for label in y_test])

# Train the CNN model
model.fit(X_train, y_train, epochs=100, batch_size=32)

# Save the CNN model to file
model.save('CNN.h5')

# Evaluate the CNN model
cnn_preds = model.predict(X_test)
cnn_preds = np.round(cnn_preds).astype(int).flatten()
cnn_rmse = mean_squared_error(y_test, cnn_preds, squared=False)
print('CNN RMSE:', cnn_rmse)

# Save KNN model to file
pickle.dump(model, open('CNN.pkl', 'wb'))
# Export model to JSON file
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)


In [ ]:
# save model ke pickle
import pickle
pickle.dump(model, open('CNN.pkl', 'wb'))

Keras weights file (<HDF5 file "variables.h5" (mode r+)>) saving:
...layers
......dense
.........vars
............0
............1
......dense_1
.........vars
............0
............1
......dense_2
.........vars
............0
............1
...metrics
......mean
.........vars
............0
............1
......mean_metric_wrapper
.........vars
............0
............1
...vars
Keras model archive saving:
File Name                                             Modified             Size
variables.h5                                   2023-04-19 22:07:40        35392
config.json                                    2023-04-19 22:07:40         2093
metadata.json                                  2023-04-19 22:07:40           64
